In [2]:
from matplotlib import colors
#import Small_Functions as sf
import numpy as np
from tkinter import filedialog
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
################################################
#First of all we load all the data. For that, we ask to the user where are the measure files of the blue cell and the red cell, 
#read each rainbow measure for each measured cell and save it on an array (BlueCell_megaarray, RecCell_megaarray) and its corresponding _names in a list.

 
BlueCell_File_Directory = os.path.join(filedialog.askdirectory(title='Select the directory of the BLUE CELL', initialdir=os.getcwd(), mustexist=True),'TXT files')   # Ask for the directory where all the EQE and JV curve files are                 #Just adding the TXT file at the end of the path because there is an extra folder to open
BlueCell_Available_folders = os.listdir(BlueCell_File_Directory)
BlueCells_names = []                                                                    #initialize the list of names   
for folder in BlueCell_Available_folders:               
    if not folder.endswith(".txt"):                                                     #find for all the folders inside the specified path whic corresponds to the measured cells without reading the calibration files (which are .txt files)
        
        cell_pathlist = os.listdir(os.path.join(BlueCell_File_Directory,folder))        #enter to each foler corresponding to each measured cell
        for file in cell_pathlist:                                                      
            if file.endswith(".txt"):                                                   #And store the data in the corresponding megaarray        
                load_txt_filepath = os.path.join(BlueCell_File_Directory,folder,file)
                actual_cell_data = np.loadtxt(load_txt_filepath, skiprows=1)
                BlueCells_names.append(folder)
                try: 
                    BlueCell_megaarray
                except NameError: 
                    BlueCell_megaarray = actual_cell_data
                else: 
                    BlueCell_megaarray = np.dstack((BlueCell_megaarray, actual_cell_data))
                
#This part is doing the same as before, but for the red cell.               
RedCell_File_Directory = os.path.join(filedialog.askdirectory(title='Select the directory of the RED CELL', initialdir=os.getcwd(), mustexist=True), 'TXT files')  # Ask for the directory where all the EQE and JV curve files are
RedCell_Available_folders = os.listdir(RedCell_File_Directory)
RedCells_names = []
for folder in RedCell_Available_folders:
    if not folder.endswith(".txt"):
        
        cell_pathlist = os.listdir(os.path.join(RedCell_File_Directory,folder))
        for file in cell_pathlist:
            if file.endswith(".txt"):
                load_txt_filepath = os.path.join(RedCell_File_Directory,folder,file)
                actual_cell_data = np.loadtxt(load_txt_filepath, skiprows=1)
                RedCells_names.append(folder)
                try: 
                    RedCell_megaarray
                except NameError: 
                    RedCell_megaarray = actual_cell_data
                else: 
                    RedCell_megaarray = np.dstack((RedCell_megaarray, actual_cell_data))

In [5]:
#Here we make the arrays corresponding to all the possible sums between cells. 
Sums_names = []                                                                     #first start the list of cell names for each sum
for i in range(len(BlueCell_megaarray[0,0,:])):                                  #Loop for the blue cell           
    for j in range (len(RedCell_megaarray[0,0,:])):                              #Loop for the red cell
        Summ_actual_array = np.zeros(RedCell_megaarray[:,:,j].shape)                            #first initialize the numpy array of the actual summ

        for r_wl_ind in range (len(RedCell_megaarray[:,0,j])):                                        #This small part is added just in case that red and blue sweep does not have the same cutting wavelenght order (it seareches for the corresponding cutting wavelenght of the red array to the blue array and makes the summ)
            b_wl_ind = np.where(BlueCell_megaarray[:,0,i]==RedCell_megaarray[r_wl_ind,0,j])           #Find the index of the blue cell array that has the same dividing wavelength as the red cell array
            Summ_actual_array [r_wl_ind,:] = BlueCell_megaarray[b_wl_ind,:,i] + RedCell_megaarray[r_wl_ind,:,j]    #For that wavelength, sum of all the parameters    
            Summ_actual_array [r_wl_ind,0:1] = Summ_actual_array [r_wl_ind,0:1]/2                     #Except the dividing wavelength (which should be the same for all the data) and sun power (which will be similar, so we make the average)
        
        Summ_actual_name = BlueCells_names[i] +'Blue cell / ' + RedCells_names[j] + 'Red cell'  #Saving the name of the corresponding cell pairs
        Summ_actual_index = np.array([i, j])                                        #and the index of the BlueCell and RedCell megaarrays so afterwards se can plot them
        
        if i==0 and j==0:                                                             #Store the data in new arrays for the first iteration
            Summ_megaarray = Summ_actual_array
            Sums_names.append(Summ_actual_name)
            Summs_indexes = Summ_actual_index
           
        else:
            Summ_megaarray = np.dstack((Summ_megaarray, Summ_actual_array))            #and stacking them for the nexts iterations
            Sums_names.append(Summ_actual_name)
            Summs_indexes = np.vstack((Summs_indexes, Summ_actual_index))

IndexError: list index out of range